In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc
from tqdm import tqdm
from sklearn import preprocessing
from scipy.stats import entropy, kurtosis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
path = 'D:/data/'
train_jet = pd.read_feather(path+'jet_complex_data/complex_train_R04_jet.feather')
train_particle = pd.read_feather(path+'jet_complex_data/complex_train_R04_particle.feather')
test_jet = pd.read_feather(path+'jet_complex_data/complex_test_R04_jet.feather')
test_particle = pd.read_feather(path+'jet_complex_data/complex_test_R04_particle.feather')

In [4]:
train_particle
train_jet

,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,jet_id
0,-211,-1.299730,0.368729,-0.057008,1.359400,0.13957,01291194f90c44c7bd79d9dbd50abd93
1,-2212,1.305420,0.418302,-0.426083,1.714930,0.93827,7df78235aa464796bbb386765d4784b5
2,22,-0.611265,0.111135,-0.225794,0.661044,0.00000,9aeaad70b98f4bf0890734137f88c7ac
3,211,3.778570,0.205956,-1.089380,3.940330,0.13957,4261ccf933b64ce6ba83c623e9a5bf06
4,22,0.461909,0.330447,-0.168640,0.592448,0.00000,e79cf654192e4cab8b83f50554fededc
...,...,...,...,...,...,...,...
24297347,22,-1.173750,0.414522,-0.616230,1.388980,0.00000,300f47688ccc47c2a59c7c674f8d5a46
24297348,211,-1.216210,0.033404,0.208287,1.242230,0.13957,853974ded875499597a0c160b718a5bd
24297349,22,2.128180,-0.497285,-1.109510,2.451010,0.00000,3104af57d9524ec9ae3fb3bfc326cd8a
24297350,211,-9.644300,4.915750,-1.419890,10.918500,0.13957,a99480901a29441d845efade2381f59a


,jet_id,number_of_particles_in_this_jet,jet_px,jet_py,jet_pz,jet_energy,jet_mass,event_id,label
0,17a0a1f19a7f43558cb9cca517b6e198,21,-288.511993,0.302560,1.839620,289.066010,17.781099,410e86bc0457482d9d6f81cd2d95ccf0,1
1,36d1005f56d343d5a09513cb7a4ead61,26,-158.447998,2.132650,-16.283400,159.718002,11.584800,6f7bf9104f8c4a619eedd129d5ce3dcb,5
2,a82071a653304481979b5dfb8eb0da47,27,-104.142998,-37.267200,-20.658501,113.320000,13.421700,4ee694b125c944c1a4560a20b79ccbe3,4
3,d3b2474fe3ba43e2b02b918adf8bbd98,17,284.808990,1.313630,0.562324,286.062012,26.710400,a0543408ffe140979e03b063fd7129c5,1
4,e46253f34c93428cb91fe0c7fc03c430,12,-38.306702,15.601100,-7.581700,42.200699,3.551420,a4b2679fd14a418c8d21fcaf2e32be8e,4
...,...,...,...,...,...,...,...,...,...
1134550,31d3749efd0d420ab1a60300e796c486,12,64.588402,5.752170,0.621394,65.040398,5.012000,1964346ac4284d628ac570189f376fd8,4
1134551,911d3d698e82491ca66e368cf68c090b,18,54.315300,-18.664900,42.585701,71.942101,7.974400,c8ca41938a264a82b04f5db5b90dca74,1
1134552,c6c67634cc3b4b0cb124fc743bb8691d,27,324.592987,-9.431880,2.120790,326.377991,32.684799,107dbefa8910492ead0ad5fe8788fc39,1
1134553,598b78dbdda94636b0b42175169f8590,12,111.586998,15.816600,38.290501,119.404999,9.458700,539ea21bad6a4f789360998e44eb91ff,4


In [4]:
#data: jet_id级样本
data = pd.concat([train_jet, test_jet], sort=False)

In [5]:
data['jet_energy/num'] = data['jet_energy'] / data['number_of_particles_in_this_jet']
data.loc[data['jet_mass']<0]['jet_mass'] = 0
data['jet_mass/num'] = data['jet_mass'] / data['number_of_particles_in_this_jet']
data['jet_px/num'] = np.abs(data['jet_px']) / data['number_of_particles_in_this_jet']
data['jet_py/num'] = np.abs(data['jet_py']) / data['number_of_particles_in_this_jet']
data['jet_pz/num'] = np.abs(data['jet_pz']) / data['number_of_particles_in_this_jet']

data['jet_|p|'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py'])+np.square(data['jet_pz']))
data['jet_pt'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py']))

data['jet_phi_cos'] = data['jet_px'] / data['jet_pt']
data['jet_eta_sinh'] = data['jet_pz'] / data['jet_pt']
data['jet_phi'] = np.arccos(data['jet_phi_cos'])
data['jet_eta'] = np.arcsinh(data['jet_eta_sinh'])

data['jet_theta_cos'] = np.tanh(data['jet_eta'])
data['jet_theta'] = np.arccos(data['jet_theta_cos'])

for col in ['jet_phi_cos', 'jet_eta_sinh', 'jet_theta_cos']:
    data[col+'_abs'] = data[col].abs()
    data = data.drop(col, axis=1)

data['jet_energy_differ'] = data['jet_energy'] - data['jet_|p|']
data['jet_energy_differ/num'] = data['jet_energy_differ'] / data['number_of_particles_in_this_jet']

data['jet_mass/pt'] = data['jet_mass'] / data['jet_pt']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
def count_all(df_group, df_merge, base_column, count_columns):
    gp = df_group.groupby(base_column, as_index=False)
    for count_column in tqdm(count_columns):
        if 'category' not in count_column:
            tp = gp[count_column].agg({
                 base_column+'_'+count_column+'_mean': 'mean', 
                 base_column+'_'+count_column+'_max': 'max', 
                 base_column+'_'+count_column+'_min': 'min', 
                 base_column+'_'+count_column+'_sum': 'sum', 
                 base_column+'_'+count_column+'_std': 'std', 
#                  base_column+'_'+count_column+'_median': 'median', 
#                  base_column+'_'+count_column+'_skew': 'skew', 
#                  base_column+'_'+count_column+'_kurt': kurtosis, 
#                  base_column+'_'+count_column+'_q1': lambda x: np.quantile(x, q=0.25), 
#                  base_column+'_'+count_column+'_q3': lambda x: np.quantile(x, q=0.75), 
        })
        else:
            tp = gp[count_column].agg({base_column+'_'+count_column+'_sum': 'sum'})
            
        df_merge = df_merge.merge(tp, on=base_column, how='left')
    return df_merge

# particle->jet

In [8]:
#data_particle: particle_id级样本
data_particle = pd.concat([train_particle, test_particle], sort=False)

In [9]:
data_particle = pd.get_dummies(data_particle, columns=['particle_category'])

In [10]:
data_particle['particle_|p|'] = np.sqrt(data_particle['particle_px']**2 + data_particle['particle_py']**2 + data_particle['particle_pz']**2)
data_particle['particle_pt'] = np.sqrt(data_particle['particle_px']**2 + data_particle['particle_py']**2)

data_particle['particle_phi_cos'] = data_particle['particle_px'] / data_particle['particle_pt']
data_particle['particle_eta_sinh'] = data_particle['particle_pz'] / data_particle['particle_pt']
data_particle['particle_phi'] = np.arccos(data_particle['particle_phi_cos'])
data_particle['particle_eta'] = np.arcsinh(data_particle['particle_eta_sinh'])

data_particle['particle_theta_cos'] = np.tanh(data_particle['particle_eta'])
data_particle['particle_theta'] = np.arccos(data_particle['particle_theta_cos'])

for col in ['particle_phi_cos', 'particle_eta_sinh', 'particle_theta_cos']:
    data_particle[col+'_abs'] = data_particle[col].abs()
    data_particle = data_particle.drop(col, axis=1)
    
data_particle['particle_energy_differ'] = data_particle['particle_energy'] - data_particle['particle_|p|']

data_particle['particle_mass/pt'] = data_particle['particle_mass'] / data_particle['particle_pt']

In [11]:
data_particle = pd.merge(data_particle, data[['event_id', 'jet_id', 'jet_phi', 'jet_eta', 'jet_pt', 'jet_energy', 'jet_theta']], on ='jet_id', how='left')

In [12]:
data_particle['particle_phi_rel'] = data_particle['particle_phi'] - data_particle['jet_phi']
data_particle['particle_eta_rel'] = data_particle['particle_eta'] - data_particle['jet_eta']
data_particle['particle_theta_rel'] = data_particle['particle_theta'] - data_particle['jet_theta']
data_particle['particle_pt_rel'] = data_particle['particle_pt'] / data_particle['jet_pt']
data_particle['particle_energy_rel'] =  data_particle['particle_energy'] / data_particle['jet_energy']
data_particle['particle_R'] = np.sqrt(data_particle['particle_phi_rel']**2 + data_particle['particle_eta_rel']**2)

In [13]:
data_particle = data_particle.drop(['jet_phi', 'jet_eta', 'jet_pt', 'jet_energy', 'jet_theta'], axis=1)

In [14]:
data.columns.to_list()
data_particle.columns.to_list()

['jet_id',
 'number_of_particles_in_this_jet',
 'jet_px',
 'jet_py',
 'jet_pz',
 'jet_energy',
 'jet_mass',
 'event_id',
 'label',
 'jet_energy/num',
 'jet_mass/num',
 'jet_px/num',
 'jet_py/num',
 'jet_pz/num',
 'jet_|p|',
 'jet_pt',
 'jet_phi',
 'jet_eta',
 'jet_theta',
 'jet_phi_cos_abs',
 'jet_eta_sinh_abs',
 'jet_theta_cos_abs',
 'jet_energy_differ',
 'jet_energy_differ/num',
 'jet_mass/pt']

['particle_px',
 'particle_py',
 'particle_pz',
 'particle_energy',
 'particle_mass',
 'jet_id',
 'particle_category_-2212',
 'particle_category_-2112',
 'particle_category_-321',
 'particle_category_-211',
 'particle_category_-13',
 'particle_category_-11',
 'particle_category_11',
 'particle_category_13',
 'particle_category_22',
 'particle_category_130',
 'particle_category_211',
 'particle_category_321',
 'particle_category_2112',
 'particle_category_2212',
 'particle_|p|',
 'particle_pt',
 'particle_phi',
 'particle_eta',
 'particle_theta',
 'particle_phi_cos_abs',
 'particle_eta_sinh_abs',
 'particle_theta_cos_abs',
 'particle_energy_differ',
 'particle_mass/pt',
 'particle_phi_rel',
 'particle_eta_rel',
 'particle_theta_rel',
 'particle_pt_rel',
 'particle_energy_rel',
 'particle_R']

In [15]:
data_particle_col = data_particle.columns.to_list()
data_particle_col.remove('jet_id', 'event_id')
data = count_all(data_particle, data, 'jet_id', data_particle_col)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [04:58<00:00,  8.52s/it]


# particle->event

# write

In [17]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32) #if not to_feather: np.float16
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [18]:
data_col = data.columns.to_list()
for col in data_col:
    if 'std' in col or 'skew' in col or 'kurt' in col:
        data[col] = data[col].fillna(0)
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672504 entries, 0 to 1672503
Data columns (total 144 columns):
jet_id                                1672504 non-null object
number_of_particles_in_this_jet       1672504 non-null int8
jet_px                                1672504 non-null float32
jet_py                                1672504 non-null float32
jet_pz                                1672504 non-null float32
jet_energy                            1672504 non-null float32
jet_mass                              1672504 non-null float32
event_id                              1672504 non-null object
label                                 1134555 non-null float64
jet_energy/num                        1672504 non-null float32
jet_mass/num                          1672504 non-null float32
jet_px/num                            1672504 non-null float32
jet_py/num                            1672504 non-null float32
jet_pz/num                            1672504 non-null float32
jet_|p|  

In [19]:
data = reduce_mem_usage(data)
data.info(verbose=True, null_counts=True)

Mem. usage decreased to 872.48 Mb (0.7% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672504 entries, 0 to 1672503
Data columns (total 144 columns):
jet_id                                1672504 non-null object
number_of_particles_in_this_jet       1672504 non-null int8
jet_px                                1672504 non-null float32
jet_py                                1672504 non-null float32
jet_pz                                1672504 non-null float32
jet_energy                            1672504 non-null float32
jet_mass                              1672504 non-null float32
event_id                              1672504 non-null object
label                                 1134555 non-null float32
jet_energy/num                        1672504 non-null float32
jet_mass/num                          1672504 non-null float32
jet_px/num                            1672504 non-null float32
jet_py/num                            1672504 non-null float32
jet_pz/num           

In [20]:
data.to_feather(path+'jet_complex_data/data_jet_level.feather')

In [21]:
data.head()

,jet_id,number_of_particles_in_this_jet,jet_px,jet_py,jet_pz,jet_energy,jet_mass,event_id,label,jet_energy/num,jet_mass/num,jet_px/num,jet_py/num,jet_pz/num,jet_|p|,jet_pt,jet_phi,jet_eta,jet_theta,jet_phi_cos_abs,jet_eta_sinh_abs,jet_theta_cos_abs,jet_energy_differ,jet_energy_differ/num,jet_mass/pt,jet_id_particle_px_mean,jet_id_particle_px_max,jet_id_particle_px_min,jet_id_particle_px_sum,jet_id_particle_px_std,jet_id_particle_py_mean,jet_id_particle_py_max,jet_id_particle_py_min,jet_id_particle_py_sum,jet_id_particle_py_std,jet_id_particle_pz_mean,jet_id_particle_pz_max,jet_id_particle_pz_min,jet_id_particle_pz_sum,jet_id_particle_pz_std,jet_id_particle_energy_mean,jet_id_particle_energy_max,jet_id_particle_energy_min,jet_id_particle_energy_sum,jet_id_particle_energy_std,jet_id_particle_mass_mean,jet_id_particle_mass_max,jet_id_particle_mass_min,jet_id_particle_mass_sum,jet_id_particle_mass_std,...,jet_id_particle_eta_sinh_abs_mean,jet_id_particle_eta_sinh_abs_max,jet_id_particle_eta_sinh_abs_min,jet_id_particle_eta_sinh_abs_sum,jet_id_particle_eta_sinh_abs_std,jet_id_particle_theta_cos_abs_mean,jet_id_particle_theta_cos_abs_max,jet_id_particle_theta_cos_abs_min,jet_id_particle_theta_cos_abs_sum,jet_id_particle_theta_cos_abs_std,jet_id_particle_energy_differ_mean,jet_id_particle_energy_differ_max,jet_id_particle_energy_differ_min,jet_id_particle_energy_differ_sum,jet_id_particle_energy_differ_std,jet_id_particle_mass/pt_mean,jet_id_particle_mass/pt_max,jet_id_particle_mass/pt_min,jet_id_particle_mass/pt_sum,jet_id_particle_mass/pt_std,jet_id_particle_phi_rel_mean,jet_id_particle_phi_rel_max,jet_id_particle_phi_rel_min,jet_id_particle_phi_rel_sum,jet_id_particle_phi_rel_std,jet_id_particle_eta_rel_mean,jet_id_particle_eta_rel_max,jet_id_particle_eta_rel_min,jet_id_particle_eta_rel_sum,jet_id_particle_eta_rel_std,jet_id_particle_theta_rel_mean,jet_id_particle_theta_rel_max,jet_id_particle_theta_rel_min,jet_id_particle_theta_rel_sum,jet_id_particle_theta_rel_std,jet_id_particle_pt_rel_mean,jet_id_particle_pt_rel_max,jet_id_particle_pt_rel_min,jet_id_particle_pt_rel_sum,jet_id_particle_pt_rel_std,jet_id_particle_energy_rel_mean,jet_id_particle_energy_rel_max,jet_id_particle_energy_rel_min,jet_id_particle_energy_rel_sum,jet_id_particle_energy_rel_std,jet_id_particle_R_mean,jet_id_particle_R_max,jet_id_particle_R_min,jet_id_particle_R_sum,jet_id_particle_R_std
0,17a0a1f19a7f43558cb9cca517b6e198,21,-288.511993,0.30256,1.839620,289.066010,17.781099,410e86bc0457482d9d6f81cd2d95ccf0,1.0,13.765048,0.846719,13.738667,0.014408,0.087601,288.518036,288.512146,3.140557,0.006376,1.564420,0.999999,0.006376,0.006376,0.547974,0.026094,0.061630,-13.738700,-0.092363,-146.042999,-288.512695,32.159996,0.014408,0.785048,-0.610681,0.302561,0.319108,0.087601,0.969680,-0.570297,1.839623,0.302749,13.772425,146.046005,0.095254,289.220917,32.148827,0.114773,0.93957,0.0,2.410240,0.222063,...,0.088200,0.321242,0.001227,1.852198,0.099020,0.086339,0.305848,0.001227,1.813126,0.095892,0.010053,0.136289,-0.000626,0.211117,0.030086,0.056251,0.466360,0.0,1.181262,0.120566,-0.104815,-0.003304,-0.323539,-2.201106,0.105882,0.020554,0.309583,-0.227097,0.431627,0.129940,-0.020322,0.225326,-0.304453,-0.426764,0.128766,0.047662,0.506201,0.000322,1.000896,0.111455,0.047645,0.505234,0.000330,1.000536,0.111216,0.142401,0.355421,0.004471,2.990414,0.137049
1,36d1005f56d343d5a09513cb7a4ead61,26,-158.447998,2.13265,-16.283400,159.718002,11.584800,6f7bf9104f8c4a619eedd129d5ce3dcb,5.0,6.143000,0.445569,6.094154,0.082025,0.626285,159.296783,158.462357,3.128132,-0.102579,1.673196,0.999909,0.102759,0.102221,0.421219,0.016201,0.073108,-6.094159,-0.261038,-26.073500,-158.448120,6.783796,0.082025,0.885232,-0.624319,2.132648,0.329125,-0.626286,0.061388,-3.275600,-16.283442,0.817435,6.145245,26.237400,0.263444,159.776352,6.829412,0.136642,0.93957,0.0,3.552680,0.245883,...,0.101645,0.208076,0.013674,2.642768,0.048130,0.100793,0.203713,0.013672,2.620626,0.047261,0.003404,0.042799,-0.00008

# analyse